In [1]:
import boto3
import sagemaker
import time
from time import strftime

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sm_client = boto3.client("sagemaker")
region = boto_session.region_name
bucket = sagemaker_session.default_bucket()
role = 'sagemaker-role'
account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]

prefix = 'sagemaker-intel-dvc'

# print(f"account: {account}")
# print(f"bucket: {bucket}")
# print(f"region: {region}")
# print(f"role: {role}")

In [2]:
from sagemaker.pytorch import PyTorchPredictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

In [3]:
predictor = PyTorchPredictor(
    endpoint_name="emlo-s13a-prod",
    sagemaker_session=sagemaker_session,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

In [4]:
classnames = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [5]:
import torch

import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np

from PIL import Image

In [6]:
transforms = T.Compose([
    T.ToTensor(),
    T.Resize((224, 224)),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
import os
directory = 'img'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    inp_img1 = Image.open(f)
    inp_img1.resize((224, 224))
    inp_img1 = transforms(inp_img1)
    input_array1 = {"inputs": inp_img1[None, ...].numpy().tolist()}
    out = predictor.predict(input_array1)
    out_t = torch.tensor(out)
    print(f"{filename} is predicted as {classnames[torch.argmax(out_t, dim=-1)[0]]}")
    print('------------------')

glacier2.jpg is predicted as glacier
------------------
forest2.jpg is predicted as forest
------------------
buildings1.jpg is predicted as buildings
------------------
mountain2.jpg is predicted as mountain
------------------
glacier1.jpg is predicted as glacier
------------------
sea1.jpg is predicted as sea
------------------
street2.jpg is predicted as street
------------------
sea2.jpg is predicted as sea
------------------
buildings2.jpg is predicted as buildings
------------------
forest1.jpg is predicted as forest
------------------
street1.jpg is predicted as street
------------------
mountain1.jpg is predicted as mountain
------------------
